In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import utils as ut

In [92]:
def reconstruct_symmetric_matrix(size, upper_triangle_array, diag=1):
    '''
    Reconstruct symmetric matrix from the upper triangle array
    '''
    # Create an empty matrix filled with zeros
    result = np.zeros((size, size))

    # Fill the upper triangle of the matrix (including the diagonal) with the values from the input array
    result[np.triu_indices_from(result)] = upper_triangle_array

    # Make the matrix symmetric by adding its transpose
    result = result + result.T

    # Fill the diagonal with the specified diagonal value (default is 1)
    np.fill_diagonal(result, diag)

    return result

In [121]:
df = pd.read_csv('/Users/rodrigo/Post-Grad/connectivity_matrices_adhd.csv')

merged_df = df.iloc[:, -36:]
y = df.iloc[:, -36:]['Age']
X_fmri = pd.DataFrame()
for i in range(df.shape[0]):
    aux = ut.remove_triangle(pd.DataFrame(reconstruct_symmetric_matrix(333, df.iloc[i,1:-36])))
    X_fmri = pd.concat([X_fmri, pd.DataFrame(aux)], axis=0)
    
X_fmri = X_fmri.reset_index().drop(columns='index')
# Identify columns with numerical names
numerical_columns = [col for col in df.columns if isinstance(col, (int, float)) or (isinstance(col, str) and col.isdigit())]

# Drop columns with numerical names
df = pd.concat([X_fmri, df.iloc[:,1:].drop(columns=numerical_columns)],axis=1)

df = df.dropna(subset='Subject')
df['Subject'] = df['Subject'].astype(int).astype(str)
df = df.set_index('Subject')

phenotypic = pd.read_csv(r'/Users/rodrigo/Post-Grad/CC400/phenotypic200.csv',
                         index_col=['Institution','Subject'])
motion = pd.read_csv(r'/Users/rodrigo/Post-Grad/CC400/motion200.csv',
                     index_col=['Institution','File'])

motion = motion.reset_index()
motion['Subject'] = motion['File'].str.extract('(\d+)')
motion['Run'] =  (motion['File'].str[-4:]).str[0]
motion['Subject'] = motion['Subject'].str.replace(r'^(0+)', '').fillna('0')
motion = motion.set_index('Subject')
motion = motion[motion['File'].str.contains('rest_1')]

motion = motion.loc[df.index,:]
df['Max Motion (mm)'] = motion['Max Motion (mm)'] 
df[' Max Rotation (degree)'] = motion[' Max Rotation (degree)'] 

/var/folders/24/hz8c6rbx3r32smymhrvkwbmc0000gn/T/ipykernel_75011/2075327235.py:1: DtypeWarning: Columns (55613,55620) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/rodrigo/Post-Grad/connectivity_matrices_adhd.csv')
/var/folders/24/hz8c6rbx3r32smymhrvkwbmc0000gn/T/ipykernel_75011/2075327235.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  motion['Subject'] = motion['Subject'].str.replace(r'^(0+)', '').fillna('0')


In [123]:
df.to_csv('/Users/rodrigo/Post-Grad/adhd_dataset_gordon.csv')